<div align=center>

# Principles of Simulation: Assignment 2

By Hamed Araab & Shahriar Khalvati

</div>


### Prerequisites

In this section, we import necessary libraries and modules required for the
execution of subsequent code cells:


In [ ]:
import pandas as pd
import seaborn as sns

from framework import *
from __future__ import annotations

### Problem 1


In [5]:
import Framework.tests.ks as ks
import Framework.generators.number.clcg as clcg

#### Generating Random Numbers

In [6]:
a1 = 40014
m1 = 2147483563
a2 = 40692
m2 = 2147483399

randomNumbers = clcg.generate(a1, m1, a2, m2, int(1e6))

print(randomNumbers[:100])

[0.56063912 0.78178844 0.85980201 0.14031065 0.47910764 0.18101787
 0.9128697  0.57077309 0.92506822 0.25080776 0.20592685 0.8697508
 0.29647089 0.51674519 0.09011976 0.69683768 0.28363259 0.70022235
 0.09757896 0.16305358 0.45806535 0.68296219 0.79065933 0.12956687
 0.98709899 0.01568144 0.10559125 0.70211277 0.01831477 0.99069293
 0.9296515  0.7854055  0.55955385 0.15767727 0.51914878 0.39857669
 0.06630876 0.80385332 0.86243963 0.01271931 0.95780907 0.15846554
 0.38492577 0.66858097 0.79403299 0.40858859 0.94024291 0.46445737
 0.04688066 0.90617129 0.11683077 0.26963024 0.0730486  0.27118083
 0.84030439 0.38189697 0.48497382 0.39756364 0.05741864 0.59236379
 0.41336667 0.18408965 0.18116575 0.92623735 0.27023931 0.13476713
 0.53985275 0.2471223  0.1259875  0.23540069 0.52211469 0.44796447
 0.22876323 0.51372436 0.14985236 0.05227165 0.46945738 0.3396017
 0.40522564 0.00797951 0.17469589 0.28751296 0.00764469 0.43145789
 0.17270577 0.04286863 0.44232397 0.53233534 0.23412355 0.038668

#### Checking Uniformity

##### Kolmogorov-Smirnov Test

In [ ]:
alpha = 0.05
isUniformKS = ks.isUniform(randomNumbers[:100], alpha)
print(isUniformKS)

##### Chi-square Test

#### Checking Independency

##### Auto-Correlation Test

### Problem 2


#### Customer


In [ ]:
class Customer:
    def __init__(self) -> None:
        self.systemArrival: float
        self.server: Server | None = None
        self.serverArrival: float | None = None
        self.serviceTime: float | None = None
        self.departure: float

    @property
    def didReturn(self) -> bool:
        return (
            self.server == None
            and self.serverArrival == None
            and self.serviceTime == None
        )

    @property
    def didWaitOutside(self) -> bool:
        return self.waitingTime["outside"] not in [None, 0]

    @property
    def waitingTime(self) -> Dict[Literal["outside", "inside"], None | float]:
        return {
            "outside": (
                None if self.didReturn else self.serverArrival - self.systemArrival
            ),
            "inside": (
                None
                if self.didReturn
                else self.departure - self.serviceTime - self.serverArrival
            ),
        }

#### Server


In [ ]:
class Server:
    def __init__(self, id: int, controller: ProblemController) -> None:
        self.id = id
        self.controller = controller
        self.status: Literal["available", "busy"] = "available"
        self.queue: List[Customer] = []

    @property
    def customersServed(self) -> List[Customer]:
        return [
            customer
            for customer in self.controller.customersServed
            if customer.server == self
        ]

    @property
    def utilizationPercentage(self) -> float:
        return (
            sum(customer.serviceTime for customer in self.customersServed)
            / self.controller.clock
        )

    @property
    def totalCustomersServed(self) -> int:
        return len(self.customersServed)

    @property
    def averageServiceTime(self) -> float:
        return sum(customer.serviceTime for customer in self.customersServed) / len(
            self.customersServed
        )

    @property
    def averageWaitingTime(self) -> float:
        return sum(
            customer.waitingTime["inside"] for customer in self.customersServed
        ) / len(self.customersServed)

#### Controller


In [ ]:
class ProblemController(SimController):
    def __init__(self, waitOutside: bool) -> None:
        super().__init__(stopTime=3 * 60, initialEvent=ArrivalEvent(initial=True))
        self.totalCustomersArrived: int = 0
        self.customersServed: List[Customer] = []
        self.outsideQueue: List[Customer] = []
        self.servers: List[Server] = [Server(i, controller=self) for i in range(1, 4)]
        self.waitOutside = waitOutside

    def simulate(self) -> Dict[str, float]:
        super().simulate()

        customersWaitedOutside = [
            customer for customer in self.customersServed if customer.didWaitOutside
        ]

        customersNotReturned = [
            customer for customer in self.customersServed if not customer.didReturn
        ]

        results = {
            "TCA": self.totalCustomersArrived,
            f"TCWO": len(customersWaitedOutside),
            f"AWTO": (
                sum(
                    customer.waitingTime["outside"] for customer in customersNotReturned
                )
                / len(customersNotReturned)
                if customersNotReturned
                else 0
            ),
        }

        for server in self.servers:
            results |= {
                f"UP{server.id}": server.utilizationPercentage,
                f"TCS{server.id}": server.totalCustomersServed,
                f"AST{server.id}": server.averageServiceTime,
                f"AWTI{server.id}": server.averageWaitingTime,
            }

        return results

#### Event


##### Arrival


In [ ]:
ARRIVAL_EVENT_INTERVAL: Callable[[], float] | None = None


class ArrivalEvent(SimEvent[ProblemController]):
    def __init__(self, initial: bool = False) -> None:
        super().__init__(0 if initial else ARRIVAL_EVENT_INTERVAL())

        self.customer = Customer()

    def trigger(self) -> None:
        self.controller.dispatchEvent(ArrivalEvent())

        self.customer.systemArrival = self.dueTime
        self.controller.totalCustomersArrived += 1

        if self.controller.outsideQueue:
            if self.controller.waitOutside:
                self.controller.outsideQueue.append(self.customer)
            else:
                self.customer.departure = self.dueTime
        else:
            server = min(self.controller.servers, key=lambda server: len(server.queue))

            if len(server.queue) + int(server.status == "busy") < 4:
                self.customer.server = server
                self.customer.serverArrival = self.dueTime

                if server.status == "available":
                    server.status = "busy"

                    self.controller.dispatchEvent(DepartureEvent(self.customer))
                else:
                    server.queue.append(self.customer)
            elif self.controller.waitOutside:
                self.controller.outsideQueue.append(self.customer)
            else:
                self.customer.departure = self.dueTime

##### Departure


In [ ]:
DEPARTURE_EVENT_INTERVAL: Callable[[], float] | None = None


class DepartureEvent(SimEvent[ProblemController]):
    def __init__(self, customer: Customer) -> None:
        super().__init__(DEPARTURE_EVENT_INTERVAL())

        self.customer = customer

    def trigger(self) -> None:
        self.customer.serviceTime = self.interval
        self.customer.departure = self.dueTime

        self.controller.customersServed.append(self.customer)

        if self.customer.server.queue:
            customer = self.customer.server.queue.pop(0)

            self.controller.dispatchEvent(DepartureEvent(customer))

            if self.controller.outsideQueue:
                customer = self.controller.outsideQueue.pop(0)
                customer.server = self.customer.server
                customer.serverArrival = self.dueTime

                self.customer.server.queue.append(customer)
        else:
            self.customer.server.status = "available"

#### Results


In [ ]:
ARRIVAL_EVENT_INTERVAL = lambda: DistributionFunction.uniform(0, 2)
DEPARTURE_EVENT_INTERVAL = lambda: DistributionFunction.uniform(2, 3)

allResults: List[Dict[str, float]] = []

for i in range(1000):
    results = ProblemController(waitOutside=True).simulate()

    allResults.append(results)

pd.DataFrame(allResults).mean(axis=0)

In [ ]:
ARRIVAL_EVENT_INTERVAL = lambda: DistributionFunction.uniform(0, 2)
DEPARTURE_EVENT_INTERVAL = lambda: DistributionFunction.uniform(2, 3)

allResults: List[Dict[str, float]] = []

for i in range(1000):
    results = ProblemController(waitOutside=False).simulate()

    allResults.append(results)

pd.DataFrame(allResults).mean(axis=0)

### Problem 3


In [2]:
from Framework.generators.variate import RandomVariateGenerator as Generator

In [26]:
totalWork25Days = 0

for i in range(25):
    randomForUniform = randomNumbers[i:i+1]
    randomForNormal = randomNumbers[i:i+2]
    loading = Generator(randomForUniform).Uniform(20, 30)[0]
    start = Generator(randomForNormal).Normal(16, 4)[0]
    part1 = Generator(randomForNormal).Normal(28, 12)[0]
    part2 = Generator(randomForNormal).Normal(79, 19)[0]
    part3 = Generator(randomForNormal).Normal(65, 15)[0]
    part4 = Generator(randomForNormal).Normal(63, 10)[0]
    part5 = Generator(randomForNormal).Normal(42, 11)[0]
    retrn = Generator(randomForNormal).Normal(19, 4)[0]
    office = Generator(randomForUniform).Uniform(20, 25)[0]
    total = loading + start + part1 + part2 + part3 + part4 + part5 + retrn + office
    totalWork25Days += total
    print(f"Day {i}: {total:.2f} min\n\tloading: {loading:.2f}min\n\tstart: {start:.2f}min\n\tpart1: {part1:.2f}min\n\tpart2: {part2:.2f}min\n\tpart3: {part3:.2f}min\n\tpart4: {part4:.2f}min\n\tpart5: {part5:.2f}min\n\treturn: {retrn:.2f}min\n\toffice work: {office:.2f} min")

print('------')
print(f"Total work in 25 days: {totalWork25Days:.2f} min")
print(f"Extra work in 25 days: {totalWork25Days - (8*60*25):.2f} min")
print(f"Average daily work: {totalWork25Days/25:.2f} min")

Day 0: 365.14 min
	loading: 25.61min
	start: 16.43min
	part1: 28.74min
	part2: 79.93min
	part3: 65.83min
	part4: 63.67min
	part5: 42.71min
	return: 19.43min
	office work: 22.80 min
Day 1: 373.63 min
	loading: 27.82min
	start: 16.89min
	part1: 29.55min
	part2: 80.95min
	part3: 66.73min
	part4: 64.41min
	part5: 43.48min
	return: 19.89min
	office work: 23.91 min
Day 2: 372.65 min
	loading: 28.60min
	start: 16.70min
	part1: 29.21min
	part2: 80.52min
	part3: 66.35min
	part4: 64.11min
	part5: 43.16min
	return: 19.70min
	office work: 24.30 min
Day 3: 310.54 min
	loading: 21.40min
	start: 12.07min
	part1: 21.19min
	part2: 70.44min
	part3: 57.39min
	part4: 56.79min
	part5: 35.48min
	return: 15.07min
	office work: 20.70 min
Day 4: 370.48 min
	loading: 24.79min
	start: 17.02min
	part1: 29.76min
	part2: 81.22min
	part3: 66.97min
	part4: 64.61min
	part5: 43.69min
	return: 20.02min
	office work: 22.40 min
Day 5: 389.72 min
	loading: 21.81min
	start: 19.16min
	part1: 33.47min
	part2: 85.88min
	part3: